In [44]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import tqdm
import psutil
import os

In [45]:
def calc_length(arr):
    res = 0
    for s in (0, 4):
        res += ((arr[s] - arr[s + 2]) ** 2 +
                (arr[s + 1] - arr[s + 3]) ** 2)
    return res


def min_length(arr):
    res = 100
    arr = arr[:8]
    #numpyに
    arr = np.array(arr)
    for s in (0,4):
        res = min(res, (arr[s] - arr[s + 2]) ** 2 +
                  (arr[s + 1] - arr[s + 3]) ** 2)
    print(f"\r{res}",end="")
    return res

def cherry_pick(df,threshold=10):
    # data[:8]のmin_lengthが10未満のものをdrop
    df = df.apply(lambda x: min_length(x) >= threshold, axis=0)



In [46]:
prefix = "0118final"
for i in range(10):
    filename = f"{prefix}/{prefix}_{i}.csv"
    # csvファイルの読み込み
    if i == 0:
        df = pd.read_csv(filename, header=0)
        df = df.drop("Unnamed: 0", axis=1)
    else:
        tmp = pd.read_csv(filename, header=0)
        tmp = tmp.drop("Unnamed: 0", axis=1)
        df = pd.concat([df, tmp], axis=0)

#cherry_pick(df)


#dataframeをnumpy配列に変換
data = df.values
#深層学習の入力データと出力データに分ける
# 入力データ
len_test = 8
x = data[:, len_test:]

#x[i]をN×Nに変換
siz = 32
x = x.reshape(-1, siz,siz)

#xは現在グレースケールなので、RGBに変換
x = np.stack((x,)*3, axis=-1)

# 出力データ
y = data[:, :len_test]
#入力データの正規化
x = (x - x.mean()) / x.std()

#入力データの各行をN×Nに変換


#出力データのMin-Max正規化
y = (y - y.min()) / (y.max() - y.min())

#入力データの次元数
input_dim = x.shape[1]

#出力データの次元数
output_dim = y.shape[1]

#入力データの数
n = x.shape[0]

#学習データとテストデータに分ける
#学習データ
x_train = x[:int(n*0.8)]
y_train = y[:int(n*0.8)]

#テストデータ
x_test = x[int(n*0.8):]
y_test = y[int(n*0.8):]


N = siz


In [47]:

#optunaを使わないCNN
def create_model(use_pretrained=False, CNN_size=0, trainable=True):
    # モデルの定義
    if use_pretrained:
        weights = "imagenet"
    else:
        weights = None


    cnn_models = [tf.keras.applications.efficientnet.EfficientNetB0,
                tf.keras.applications.efficientnet.EfficientNetB1,
                tf.keras.applications.efficientnet.EfficientNetB2,
                tf.keras.applications.efficientnet.EfficientNetB3,
                tf.keras.applications.efficientnet.EfficientNetB4,
                tf.keras.applications.efficientnet.EfficientNetB5,
                tf.keras.applications.efficientnet.EfficientNetB6,
                tf.keras.applications.efficientnet.EfficientNetB7]

    if CNN_size <= len(cnn_models):
        base_model = cnn_models[CNN_size](
            include_top=False,
            weights=weights,
            input_shape=(N, N, 3)
        )

    base_model.trainable = trainable


    #学習済データは使用しない


    #8個の出力を持つモデル
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(N, N, 3)),
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8),
    ])
    
    # モデルのコンパイル
    model.compile(
        optimizer="Adam",
        loss="mse",
        metrics=["mae"],
    )
    return model


#学習の実行
def objective(use_pretrained=False, CNN_size=0, trainable=True):
    model = create_model(use_pretrained, CNN_size, trainable=True)
    history = model.fit(
        x_train,
        y_train,
        epochs=500,
        batch_size=32,
        validation_data=(x_test, y_test),
    )
    #学習の様子をプロット
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    return model, history

In [48]:
#片対数プロット
def plot_and_save(history,filename):
    #pltを初期化
    plt.clf()
    #plotを作成
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    #plotの上限を設定
    plt.ylim(0, 0.1)
    plt.legend()
    ax = plt.gca()

    #軸ラベルを追加
    ax.set_xlabel("epoch")
    ax.set_ylabel("loss")

    plt.show()
    plt.savefig(filename)


In [49]:

# model,history = objective()
# model2,history2 = objective(True)#use_pretrained

for CNN_size in range(8):
    for use_trained in (True,False):
        for trainable in (True,False):
            model,history = objective(use_trained)
            model.save(f"0121Project/model_{CNN_size}_{use_trained}_{trainable}.h5")
            history_df = pd.DataFrame(history.history)
            history_df.to_csv(f"0121Project/history_{CNN_size}_{use_trained}_{trainable}.csv")
            #メモリ解放
            del model
            del history
            del history_df
            # メモリ使用量の確認
            print("memory usage: ", psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024, "MB")
            filename = f"0121Project/plot_{CNN_size}_{use_trained}_{trainable}.png"
            print(filename)
            plot_and_save(history,filename)

Epoch 1/500
250/250 [==============================] - 22s 60ms/step - loss: 0.2452 - mae: 0.3558 - val_loss: 0.1148 - val_mae: 0.2828
Epoch 2/500
250/250 [==============================] - 13s 53ms/step - loss: 0.0731 - mae: 0.2199 - val_loss: 0.1143 - val_mae: 0.2842
Epoch 3/500
250/250 [==============================] - 13s 54ms/step - loss: 0.0659 - mae: 0.2070 - val_loss: 0.0976 - val_mae: 0.2649
Epoch 4/500
250/250 [==============================] - 13s 53ms/step - loss: 0.0593 - mae: 0.1941 - val_loss: 0.0967 - val_mae: 0.2645
Epoch 5/500
250/250 [==============================] - 14s 56ms/step - loss: 0.0552 - mae: 0.1862 - val_loss: 0.0973 - val_mae: 0.2641
Epoch 6/500
250/250 [==============================] - 16s 64ms/step - loss: 0.0501 - mae: 0.1753 - val_loss: 0.1024 - val_mae: 0.2666
Epoch 7/500
250/250 [==============================] - 17s 68ms/step - loss: 0.0462 - mae: 0.1674 - val_loss: 0.1083 - val_mae: 0.2741
Epoch 8/500
250/250 [==============================] - 

KeyboardInterrupt: 